In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

<h3>Предобработка данных</h3>

In [20]:
df = pd.read_csv('data\data_preprocessed.csv')

In [21]:
df = df[df['bundle_nm'].isin(['Без подписки', 'Pro'])]
df['pro_flg'] = df['bundle_nm'].map({'Без подписки': 0, 'Pro': 1})

In [22]:
df.dropna(subset=['age', 'gender_cd', 'region_size'], inplace=True)
df.drop_duplicates(subset=['customer_id'], inplace=True, keep='first') # дубликаты клиентов
IQR = df["state_talk_time_sec"].quantile(0.75) - df["state_talk_time_sec"].quantile(0.25)
high = df["state_talk_time_sec"].quantile(0.75) + 3 * IQR
df = df[(df["state_talk_time_sec"] < high) & (df["state_talk_time_sec"] > 0)] # чистка длительности звонка

In [23]:
df_filter = df[(df['age'] >= 14) & (df['age'])]
df_filter[df_filter['product'] == 'Кредит']['product'] = df_filter[df_filter['age'] >= 18]['product']
df_filter[df_filter['product'] == 'Кредитная карта']['product'] = df_filter[df_filter['age'] >= 18]['product']
df_filter = df_filter[df_filter["foreign_flg"] == 0]

In [24]:
df_filter.to_csv('data/data_final_preprocessed.csv', index=False)